# Ex ? Manipulating Spectra Part 2 - `Spectra`

## Notebook Initialization

In [2]:
%load_ext autoreload
import sys
sys.path.append("..")

In [3]:
%matplotlib inline

In [10]:
%autoreload


import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

import numpy as np
from scipy.integrate import quad

import darkhistory.physics as phys
import darkhistory.spec.spectools as spectools
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra

In [15]:
eng = np.array([1, 10, 100, 1000])
spec_arr = [Spectrum(eng, np.ones(4)*i, rs=100, spec_type='N') for i in np.arange(4)]
test_spectra = Spectra(spec_arr)
test_spectra.redshift(np.array([0.01, 0.1, 1, 10]))
print(test_spectra.grid_vals)

[100 100 100 100]
100
0.01
[100 100 100 100]
100
0.1
[100 100 100 100]
100
1.0
[100 100 100 100]
100
10.0
[[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [2. 2. 0. 0.]
 [3. 3. 3. 0.]]


In [12]:
np.ones(4)

array([1., 1., 1., 1.])